In [2]:
import pandas as pd

In [108]:
df_nisqa = pd.read_csv("results/NISQA_results.csv")
df_nisqa.reset_index()
df_nisqa.index = range(df_nisqa.shape[0])
df_splitter = pd.read_csv("df.csv")
df_splitter = df_splitter.astype({'filename': 'string','transcript':'string', 'accent':'string' })
df_nisqa["filename"] = df_nisqa["deg"]
df_nisqa = df_nisqa.drop(["deg"],axis=1)

split_params = ['duration','transcript','confidence','age','gender','accent']
# csv_nisqa_params = ['mos_pred', 'nos_pred', 'dis_pred', 'col_pred','loud_pred', 'model']
for fields in split_params:
    df_nisqa[fields] = ''

In [ ]:
counter = 0
for i in df_nisqa.index:
    from math import ceil
    parent_audio_part = df_nisqa.loc[i, "filename"].split("_")[0:2] #using this because chunks are seperated by _
    # parent_audio_part[1] = parent_audio_part[1][:-4] #this cuts off the .wav file extension This will be commented in final code as last part will contain extension and we dont need it call_0 -> needed "_1.wav" -> discard
    parent_audio = "_".join(parent_audio_part)
    original_params = df_splitter.loc[df_splitter["filename"] == parent_audio]
    transcript = original_params.loc[:, "transcript"].split()
    original_params = df_splitter.loc[df_splitter["filename"] == parent_audio]
    duration = original_params["duration"]
    no_of_subaudios = ceil(duration/5)
    chunk_size = ceil(len(transcript)/no_of_subaudios)
    list_chunked = [transcript[j:j + chunk_size] for j in range(0, len(transcript), chunk_size)]
    sentence = []
    for word in list_chunked:
        sentence.append(" ".join(word))
    
    if parent_audio in df_nisqa.loc[i,"filename"]:
        same_flag = True
        # insert transcript here 
        # i can use try except block here for handling list index out of range
        try:
            df_nisqa.loc[i,"transcript"] = sentence[counter]
        except IndexError:
            df_nisqa.loc[i,"transcript"] = ""
        same_flag = False
        counter+=1
        # except NameError:
        #     counter = 0
    else:
        same_flag = False
        counter = 0
    # break




In [73]:
df_nisqa = pd.read_csv("results/NISQA_results.csv")
df_nisqa.reset_index()
df_nisqa.index = range(df_nisqa.shape[0])

In [122]:
df_nisqa.loc[1, "transcript"]

''

In [21]:
from math import ceil
df_nisqa = pd.read_csv("results/NISQA_results.csv")
df_nisqa.reset_index(inplace=True)
df_nisqa.index = range(df_nisqa.shape[0])
df_splitter = pd.read_csv("df.csv")
df_splitter = df_splitter.astype({'filename': 'string','transcript':'string', 'accent':'string' })
df_nisqa["filename"] = df_nisqa["deg"]
df_nisqa = df_nisqa.drop(["deg"],axis=1)
df_new = pd.DataFrame()

# split_params = ['duration','transcript','confidence','age','gender','accent']
csv_nisqa_params = ['filename', 'duration', 'transcript', 'confidence', 'age', 'gender','accent' ,'mos_pred', 'noi_pred', 'dis_pred', 'col_pred','loud_pred', 'model']
for fields in csv_nisqa_params:
    df_new[fields] = ''
extra_nisqa_params = ['filename', 'duration', 'transcript', 'confidence', 'age', 'gender','accent']

def make_chunks(transcript: str, chunk_length: int, number_of_chunks: int):
    return [transcript[i * chunk_length:(i + 1) * chunk_length]
            for i in range(int(number_of_chunks))]

In [22]:
for i in df_splitter.index:
    # print("iteration ", i)
    parent_params = df_splitter.loc[i]
    filename = parent_params.loc["filename"]
    concerned_params = df_splitter.loc[i, ["filename", "duration","confidence", "age", "gender", "accent"]]
    transcript = df_splitter.loc[i, "transcript"]
    chunks = df_nisqa[df_nisqa["filename"].str.contains(filename)]
    chunks.reset_index(inplace=True)
    for fields in extra_nisqa_params:
        chunks[fields] = ''
    no_of_sub_calls = chunks.shape[0]
    chunk_length = ceil(len(transcript)/no_of_sub_calls)
    # print(no_of_sub_calls)
    sub_transcripts = make_chunks(transcript, chunk_length, no_of_sub_calls)
    # print(sub_transcripts)
    # print(len(chunks.index), "\n",len(sub_transcripts))
    # break
    # print(i)
    for sub_chunks in chunks.index:
        # print("Sub_ ITERATION ", sub_chunks)
        # print(sub_chunks)
        # print(chunks.loc[sub_chunks, "transcript"])
        chunks.loc[sub_chunks, "transcript"] = sub_transcripts[sub_chunks]
        # final_parameters = pd.concat([concerned_params, chunks.loc[sub_chunks]], axis=1, ignore_index=True)
        # print({'filename': df_nisqa.loc[sub_chunks, "filename"], 'duration': df_splitter.loc[i, "duration"], 'confidence':concerned_params.loc["confidence"],'age':concerned_params.loc["age"],'gender': concerned_params.loc["gender"], 'accent': concerned_params.loc["accent"],'mos_pred': chunks.loc[sub_chunks, "mos_pred"], 'noi_pred':chunks.loc[sub_chunks, "noi_pred"], 'dis_pred': chunks.loc[sub_chunks, "dis_pred"], 'col_pred':chunks.loc[sub_chunks, "col_pred"], 'loud_pred':chunks.loc[sub_chunks, "loud_pred"], 'model':chunks.loc[sub_chunks, "model"], 'transcript':chunks.loc[sub_chunks, "transcript"]})
        df_new = df_new.append({'filename': df_nisqa.loc[sub_chunks, "filename"], 'duration': df_splitter.loc[i, "duration"], 'confidence':concerned_params.loc["confidence"],'age':concerned_params.loc["age"],'gender': concerned_params.loc["gender"], 'accent': concerned_params.loc["accent"],'mos_pred': chunks.loc[sub_chunks, "mos_pred"], 'noi_pred':chunks.loc[sub_chunks, "noi_pred"], 'dis_pred': chunks.loc[sub_chunks, "dis_pred"], 'col_pred':chunks.loc[sub_chunks, "col_pred"], 'loud_pred':chunks.loc[sub_chunks, "loud_pred"], 'model':chunks.loc[sub_chunks, "model"], 'transcript':chunks.loc[sub_chunks, "transcript"]}, ignore_index=True)

    # break
        # print(final_parameters)
        # df_new = df_new.append(final_parameters, ignore_index=True)
    
df_new = df_new.sort_values('noi_pred')
df_new.reset_index(inplace=True)
df_new.drop(["index"], axis=1, inplace=True)
df_new.to_csv('final_data.csv')

    


C:\Users\Himanshu\AppData\Local\Temp\ipykernel_8640\1066049726.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunks[fields] = ''
c:\Users\Himanshu\anaconda3\envs\nisqa\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [23]:
df_new

,filename,duration,transcript,confidence,age,gender,accent,mos_pred,noi_pred,dis_pred,col_pred,loud_pred,model
0,OutboundSampleRecording_0_2,22,I am her to make yo,-3.789234,0,0,0,2.581293,1.521738,4.343019,3.584870,3.764795,NISQAv2
1,OutboundSampleRecording_0_1,6,Hello tre,-3.312910,0,0,0,1.809590,1.875911,3.833058,3.287724,2.815992,NISQAv2
2,OutboundSampleRecording_0_2,6,my love,-3.312910,0,0,0,1.742911,1.892213,3.259427,2.251022,2.962693,NISQAv2
3,OutboundSampleRecording_0_4,22,then I qwill carry ea,-3.789234,0,0,0,2.635692,1.920747,3.986793,3.092535,3.441731,NISQAv2
4,OutboundSampleRecording_0_1,5,see the ocean ovwer there,-15.193325,0,0,0,2.008507,1.956141,3.775151,3.138167,2.895992,NISQAv2
5,OutboundSampleRecording_0_5,22,ch of you to the back,-3.789234,0,0,0,2.189593,2.064310,3.807749,3.011560,3.009693,NISQAv2
6,OutboundSampleRecording_0_1,4,I am going to the movies do you want to go,-6.178956,0,0,0,2.058475,2.106824,3.524250,2.770824,3.104320,NISQAv2
7,OutboundSampleRecording_0_1,12,I am going to t,-3.789234,0,0,0,2.361769,2.156052,3.397645,3.008578,3.257230,NISQAv2
8,OutboundSampleRecording_0_2,12,he movies do yo,-3.789234,0,0,0,2.016181,2.240076,4.008885,3.345069,3.415656,NISQAv2
9,OutboundSampleRecording_0_3,22,u my aquaintance and,-3.789234,0,0,0,2.475892,2.258558,3.674884,3.071306,3.260052,NISQAv2


In [219]:
pd.concat([concerned_params, chunks.loc[sub_chunks]], axis=1, ignore_index=True)

,0,1
filename,OutboundSampleRecording_4,
duration,12,
confidence,-3.789234,
age,0,
gender,0,
accent,0,
level_0,NaN,11
index,NaN,11
mos_pred,NaN,2.172753
noi_pred,NaN,2.426229


In [213]:
pd.merge(concerned_params, chunks[sub_chunks], how='left', )

filename      OutboundSampleRecording_4
duration                             12
confidence                    -3.789234
age                                   0
gender                                0
accent                                0
Name: 4, dtype: object

In [224]:
concerned_param

filename      OutboundSampleRecording_4
duration                             12
confidence                    -3.789234
age                                   0
gender                                0
accent                                0
Name: 4, dtype: object